# Prep DF If needed

In [1]:
%%script false --no-raise-error

# Loading in Data (skip this cell if already done)
import re
import pandas as pd

def preprocess_quote(quote):
    # Replace missing spaces with space
    preprocessed_quote = re.sub(r"(?<=[a-z])(?=[A-Z])", " ", quote)

    # ...
    # add more if necessary

    return preprocessed_quote

df = pd.read_excel("data/qadata.xlsx")
df[["name", "party"]] = df.username.str.split(" - ", expand=True).apply(
    lambda x: x.str.strip()
)
df = df.drop(["username"], axis=1)

df["answer"] = df["answer"].apply(preprocess_quote)

# removing None values in "party"
df["party"] = df["party"].apply(lambda p: p if p else "Other")
df['name'] = df.name.str.replace('&quot;', '"')
df.to_pickle("./data/mod_qadata")
df.to_excel("./data/mod_qadata.xlsx", index=False)
df.sample(5)

# Do Stuff

In [2]:
import os
if not os.path.exists("./chroma_qadata_db"):
    !unzip chroma_qadata_db.zip

In [6]:
from genie_master import GenieMaster
import pandas as pd
import asyncio
import aiohttp
import time
import tqdm.asyncio
from tqdm import tqdm

US_STATES = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
    "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
    "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
    "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
    "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

db_path = "./chroma_qadata_db"
gm = GenieMaster(db_path=db_path)

if os.path.exists("./data/all_people"):
    df_people = pd.read_pickle("./data/all_people")
else:
    df = pd.read_pickle("./data/mod_qadata")
    df_people = df.drop_duplicates(subset='name')[['name', 'party', 'usertitle']]
print("# of people:", df_people.shape[0])
df_people.sample(3)

Genie Master initialized at: 2023-08-28 01:35:15.589350-04:00
# of people: 4281


,name,party,usertitle
14097,Jesse Tyler Pippy,R,House of Delegates District 4
31862,Angela Paxton,R,State Senator District 8
11432,Ashley Hinson,R,Iowa Congressional District 2


In [9]:
df_people.to_pickle("./data/all_people")

In [42]:
async def process_row(session, genie, row, output_path, container):
    output = await genie.async_ask(row["question"])
    result = output["result"]

    row['answer'] = result.get('answer', "")
    row['reasoning'] = result.get('reasoning', "")
    row['evidence'] = result.get('evidence', "")
    
    if output["source_documents"]:
        source_doc = output["source_documents"][0]
        row['source_content'] = source_doc['source_content']
        row['source_category'] = source_doc['source_category']
        row['source_sub_category'] = source_doc['source_sub_category']
    
    row['cost'] = output["total_cost"]
    new_row_df = pd.DataFrame([row])
    new_row_df.to_csv(output_path, mode='a', header=False, index=False)

    container[0] += output["total_cost"]

    return row

async def process_rows(df, genies, output_path, include_header=False):
    container = [0] # cost
    async with aiohttp.ClientSession() as session:
        tasks = []
        for _, row in df.iterrows():
            genie = genies[row["name"]]
            task = process_row(session, genie, row.copy(), output_path, container)
            tasks.append(task)
        processed_rows = await asyncio.gather(*tasks)
    
    processed_df = pd.DataFrame(processed_rows)
    # processed_df.to_csv(output_path, index=False)  # Save as CSV

    return processed_df
            
async def main(df, model_name="gpt-3.5-turbo", output_file='./result/output.csv', clear_file=True):
    genies = {name: gm.get_genie(name, model_name=model_name) for name in df.name.unique()}
    if clear_file:
        open(output_file, 'w').close() # erase content inside csv file

    RPM = 10
    # rate limit for gpt-4: 40,000 tokens/min, 200 req/min. For this purpose, 40K TPM / 750 Tokens per Requests =  ~50 RPM < 200 RPM. So let's limit it to 50 RPM
    if model_name == "gpt-4":
        RPM = 50
    # rate limit for gpt-3.5: 90,000 TPM, 3500 req/min. For this purpose, 90K/750 = 120. So let's limit it to 120
    if model_name == "gpt-3.5-turbo":
        RPM = 120
    
    batch_size = round(RPM * 0.9)

    cost_benchmarks = [0.01, 0.1, 1, 5] + list(range(10, 110, 10))
    curr_bm = 0
    curr_cost = 0

    processed_rows_list = []
    num_rows = df.shape[0]
    for batch_start in tqdm(range(0, num_rows, batch_size), desc="Processing rows", total=num_rows):
        batch_end = min(batch_start + batch_size, num_rows)
        df_batch = df.iloc[batch_start:batch_end]

        processed_rows = await process_rows(df_batch, genies, output_file)
        processed_rows_list.append(processed_rows)

        curr_cost += processed_rows['cost'].sum()
        if curr_cost.sum() > cost_benchmarks[curr_bm]:
            print(f"Cost exceeded {cost_benchmarks[curr_bm]}: {round(curr_cost, 4)}")
            curr_bm += 1

        time.sleep(60)
    new_df = pd.concat(processed_rows_list)

    print("Total cost:", new_df['cost'].sum())
    return new_df

In [5]:
def seq_process_row(row, genie):
    output = genie.ask(row["question"])
    result = output["result"]

    row['answer'] = result.get('answer', "")
    row['reasoning'] = result.get('reasoning', "")
    row['evidence'] = result.get('evidence', "")
    
    if output["source_documents"]:
        source_doc = output["source_documents"][0]
        row['source_content'] = source_doc['source_content']
        row['source_category'] = source_doc['source_category']
        row['source_sub_category'] = source_doc['source_sub_category']
    
    row['cost'] = output["total_cost"]
    
    return row

def seq_main(df, batch_size=10):
    output_excel_path = './output.xlsx'
    genies = {name: gm.get_genie(name) for name in df.name.values}
    df_results = pd.DataFrame()
    for _, row in df.iterrows():
        df_results = pd.concat([df_results, seq_process_row(row, genies[row['name']])])
        if df_results.shape[0] % batch_size == 0:
            df_results.to_excel(output_excel_path)
    df_results.to_excel(output_excel_path)
    df = df_results


# Questions

In [6]:
questions = [
    "Should the government reduce spending on medicare or social security programs?", # social security and medicare
    "Should the government forgive student loan?", # education
    "Should the government regulate what is being taught in school?", # education
    "Should abortion be legal?", # abortion
    "Should there be stricter gun control laws?", # gun control
    "Should the U.S. government maintain and possibly expand spending on foreign aid?", # foreign policy
    "Should there be stricter border control measures?", # immigration
    "Should LGBTQ issues be included in school curricula?", # LGBTQ, education
    "Do you support transgender individuals' access to gender-affirming healthcare?", # LGBTQ
    "Do you support qualified immunity for police officers?", # crime
]
questions = [q.lower() for q in questions]

In [7]:
people_tuples = df_people.values.tolist()
combinations = [tup + [q] for tup in people_tuples for q in questions]
df_all = pd.DataFrame(combinations, columns=['name', 'party', 'usertitle', 'question'])

print("Number of rows:", df_all.shape[0])
df_all.iloc[:5]

Number of rows: 42810


,name,party,usertitle,question
0,Joe Biden,D,US President,should the government reduce spending on medic...
1,Joe Biden,D,US President,should the government forgive student loan?
2,Joe Biden,D,US President,should the government regulate what is being t...
3,Joe Biden,D,US President,should abortion be legal?
4,Joe Biden,D,US President,should there be stricter gun control laws?


# Senators

In [43]:
pattern = r'^(?i)(' + '|'.join(US_STATES) + r')\s+senator$'

# Filter rows with values that match the pattern
df_senators = df_all[df_all['usertitle'].str.match(pattern, na=False)].sample(40)
print(df_senators.shape[0])
df_senators.iloc[:5]

40


,name,party,usertitle,question
12193,Jerry Moran,R,Kansas Senator,should abortion be legal?
12999,Mitch McConnell,R,Kentucky Senator,do you support qualified immunity for police o...
4757,Christopher Coons,D,Delaware Senator,should lgbtq issues be included in school curr...
32625,Jack Reed,D,Rhode Island Senator,should the u.s. government maintain and possib...
17695,Tina Smith,D,Minnesota Senator,should the u.s. government maintain and possib...


In [46]:
model_name="gpt-3.5-turbo"
output_file="./result/senators.csv"
s = time.perf_counter()
df_senators = await main(df_senators, model_name=model_name, output_file=output_file, clear_file=True)
elapsed = time.perf_counter() - s
print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

Processing rows:   0%|          | 0/4 [00:00<?, ?it/s]

Cost exceeded 0.01: 0.012


Processing rows:  25%|██▌       | 1/4 [01:07<03:23, 67.88s/it]

## clean up leftovers

In [ ]:
df_temp_senators = pd.read_csv("result/senators.csv", header=None)
df_temp_senators.columns = ["name", "party", "usertitle", "question", "answer", "reasoning", "evidence", "source_content", "source_category", "source_sub_category", "cost"]
df_temp_senators.iloc[:3]

In [32]:
# Merge the two DataFrames and mark the rows using an indicator
merged = df_senators.merge(df_temp_senators[["name", "party", "usertitle", "question"]], on=['name', "party", "usertitle", 'question'], how='left', indicator=True)

# Filter the rows to keep only those not present in both DataFrames
result_df = merged[merged['_merge'] == 'left_only']

# Drop the '_merge' column, which was used for indication
result_df = result_df.drop(columns=['_merge'])
result_df.to_pickle("./data/remaining_senators")

In [ ]:
df_remaining_senators = pd.read_pickle("./data/remaining_senators")
model_name="gpt-3.5-turbo"
output_file="./result/senators.csv"
s = time.perf_counter()
df_remaining_senators = await main(df_remaining_senators, model_name=model_name, output_file=output_file, clear_file=False)
elapsed = time.perf_counter() - s
print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

# Congress

# Governor

# 

# Delete Database to save space

In [ ]:
!rm -rf ./chroma_qadata_db